In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -qr requirements.txt

In [1]:
import cv2
import torch
import tkinter as tk
from PIL import Image, ImageTk
import time

In [2]:
# tkinter 팝업 알림을 표시하는 함수 정의
def show_popup():
    # Tkinter 윈도우 생성
    root = tk.Tk()
    root.title("낙상 감지 시스템")

    # 창의 최대 크기 설정 (가로:세로 비율 4:3)
    max_width = 480
    max_height = 360
    root.maxsize(max_width, max_height)

    # 이미지 열기 (PIL로)
    image = Image.open("../배경.png")  # 이미지 파일 경로에 맞게 수정
    # 이미지 크기를 창의 최대 크기에 맞게 조정
    image.thumbnail((max_width, max_height))
    tk_image = ImageTk.PhotoImage(image)

    # 캔버스 생성
    canvas = tk.Canvas(root, width=max_width, height=max_height)
    canvas.pack()

    # 이미지를 캔버스에 배치
    canvas.create_image(0, 0, anchor="nw", image=tk_image)

    # 버튼 이미지 열기 (PIL로)
    button_image = Image.open("../확인.png")  # 버튼 이미지 파일 경로에 맞게 수정

    # 원하는 버튼 크기를 지정
    button_width = 100  # 원하는 가로 크기
    button_height = 40  # 원하는 세로 크기
    tk_button_image = ImageTk.PhotoImage(button_image)

    # 이미지 버튼 생성
    button = tk.Button(root, image=tk_button_image, command=root.destroy, width=button_width, height=button_height)
    # 버튼을 배치할 위치 설정
    button_x = (max_width - button_width) // 2  # X 좌표 중앙 정렬
    button_y = (max_height - button_height) - 35  # Y 좌표 중앙 정렬

    # 이미지를 버튼의 중앙에 위치하도록 anchor 설정
    button.config(compound="center")

    canvas.create_window(button_x, button_y, window=button, anchor="nw")

    # Tkinter 메인 루프 실행
    root.mainloop()


In [3]:
cd yolov5

C:\Users\USER\Project\yolov5


In [4]:
# YOLOv5 모델을 불러옵니다.
model = torch.hub.load('ultralytics/yolov5', 'custom', '../1026.pt')

# 웹캠 입력을 설정합니다.
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)

sit_start_time = None  # 'sit' 타이머 시작 시간
lie_start_time = None  # 'lie' 타이머 시작 시간
sleep_condition = False
falling_condition = False

while True:
    # 프레임 읽기
    ret, img = cam.read()

    # 객체 감지 수행
    result = model(img)

    # 결과에서 경계 상자 좌표를 추출합니다.
    #0:왼,위 x / 1:왼,위 y / 2:오,아 x / 3:오,아 y / 4:신뢰도 / 5:클래스 인덱스
    bboxes = result.xyxy[0].cpu().numpy()      
    class_labels = result.names

    # 이미지에 경계 상자 그리기
    for bbox in bboxes:
        class_idx = int(bbox[5])
        class_name = class_labels[class_idx]
        confidence = bbox[4]
        
        if class_name == 'stand':
            cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)  #green
            sit_start_time = None  # 'stand'이 감지되면 'sit' 타이머 초기화
            lie_start_time = None   # 'stand'이 감지되면 'lie' 타이머 초기화
            
        elif class_name == 'sit':
            cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)  #blue
            if sit_start_time is None:
                sit_start_time = time.time()
                
             #sit이 2초 미만으로 감지될 때만 falling_condition을 True로 설정
            if time.time() - sit_start_time < 2:
                falling_condition = True
            else:
                falling_condition = False

        elif class_name == 'lie':
            cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)  #red
            if falling_condition == True or sit_start_time is None:
                if lie_start_time is None:
                    lie_start_time = time.time()
                elif time.time() - lie_start_time >= 5:
                    show_popup()              
                    falling_condition = False 
                    lie_start_time = None
            
        # 클래스 이름과 신뢰도 표시
        text = f'{class_name}: {confidence:.2f}'
        cv2.putText(img, text, (int(bbox[0]), int(bbox[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
    # 화면에 표시
    cv2.imshow('Object Detection', img)

    # 'q' 또는 'Esc' 키를 눌러 종료
    if (cv2.waitKey(1) & 0xFF == ord("q")) or (cv2.waitKey(1) == 27):
        break

# 카메라 닫기
cam.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-10-9 Python-3.9.16 torch-2.1.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [6]:
##############################################신뢰도가 0.4 이상인 경우만 박스 표시######################################################

# YOLOv5 모델을 불러옵니다.
model = torch.hub.load('ultralytics/yolov5', 'custom', '../1026.pt')

# 웹캠 입력을 설정합니다.
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)

sit_start_time = None  # 'sit' 타이머 시작 시간
lie_start_time = None  # 'lie' 타이머 시작 시간
sleep_condition = False
falling_condition = False

while True:
    # 프레임 읽기
    ret, img = cam.read()

    # 객체 감지 수행
    result = model(img)

    # 결과에서 경계 상자 좌표를 추출합니다.
    #0:왼,위 x / 1:왼,위 y / 2:오,아 x / 3:오,아 y / 4:신뢰도 / 5:클래스 인덱스
    bboxes = result.xyxy[0].cpu().numpy()      
    class_labels = result.names

    # 이미지에 경계 상자 그리기
    for bbox in bboxes:
        class_idx = int(bbox[5])
        class_name = class_labels[class_idx]
        confidence = bbox[4]
        
        # 신뢰도가 0.4 이상인 경우에만 바운딩 박스 표시
        if confidence >= 0.4:
            if class_name == 'stand':
                cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)  #green
                sit_start_time = None  # 'stand'이 감지되면 'sit' 타이머 초기화
                lie_start_time = None   # 'stand'이 감지되면 'lie' 타이머 초기화

            elif class_name == 'sit':
                cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)  #blue
                if sit_start_time is None:
                    sit_start_time = time.time()

                 #sit이 2초 미만으로 감지될 때만 falling_condition을 True로 설정
                if time.time() - sit_start_time < 2:
                    falling_condition = True
                else:
                    falling_condition = False

            elif class_name == 'lie':
                cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)  #red
                if falling_condition == True or sit_start_time is None:
                    if lie_start_time is None:
                        lie_start_time = time.time()
                    elif time.time() - lie_start_time >= 5:
                        show_popup()              
                        falling_condition = False 
                        lie_start_time = None
            
            # 클래스 이름과 신뢰도 표시
            text = f'{class_name}: {confidence:.2f}'
            cv2.putText(img, text, (int(bbox[0]), int(bbox[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
    # 화면에 표시
    cv2.imshow('Object Detection', img)

    # 'q' 또는 'Esc' 키를 눌러 종료
    if (cv2.waitKey(1) & 0xFF == ord("q")) or (cv2.waitKey(1) == 27):
        break

# 카메라 닫기
cam.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-10-9 Python-3.9.16 torch-2.1.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
